# Model

The puspose of this notebook is to create a model and all the features that can help to improve the accuracy of such model.

It is been decided to use a Random Forest due to the good results it gives

We will follow the following points:

1. Create the dataframe
2. Times feature engineering
3. Lag Variables
4. Weather feature engineering
5. The function for all the feature engineering
6. Model
    1. First Model (Random Forest)
    2. RandomizedSearchCV Model
    3. Test
7. Visualization

*NOTE: Kindly note that this notebook has been "cleaned up" and all the code has been organized for your easy understanding.*

## 1. Create the dataframe

In [9]:
#Import libraries
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_columns', None)
from datetime import datetime

In [2]:
#Get the preprocessig_v2.py data path
cwd = os.getcwd()
cwd_list = cwd.split(os.sep)
preprocessing_path = os.path.join(os.sep.join(cwd_list[:-1]), '02-EDA')
preprocessing_path

'/home/gonzalo/Repos/TFM/02-EDA'

In [3]:
#Import preprocessing_v2
%cd $preprocessing_path
import preprocessing_v2 as pre
%cd $cwd

/home/gonzalo/Repos/TFM/02-EDA
/home/gonzalo/Repos/TFM/03-MODEL


**Please, define the datapath where the data have been unziped**

In [4]:
data_path = ''

In [5]:
#Define data path WINDOWS
# data_path = os.path.join(os.sep.join(cwd_list[:-1]), 'DATA')
# data_path

In [6]:
#Define data path LINUX
data_path = '/home/gonzalo/Data/TFM/DATA/'
data_path

'/home/gonzalo/Data/TFM/DATA/'

In [7]:
#Load the df_dock_bikes
df_dock_bikes = pre.make_all_dataset(data_path, 'dock_bikes',by_postal_code=True, verbose=True )

0%[--10%------------------]100%
Bicimad_Estacions_201810.json added to DataSet
0%[----20%----------------]100%
0%[------30%--------------]100%
Bicimad_Stations_201901.json added to DataSet
0%[--------40%------------]100%
0%[----------50%----------]100%


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/gonzalo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-e766cb7bf622>", line 2, in <module>
    df_dock_bikes = pre.make_all_dataset(data_path, 'dock_bikes',by_postal_code=True, verbose=True )
  File "/home/gonzalo/Repos/TFM/02-EDA/preprocessing_v2.py", line 175, in make_all_dataset
    df_aux = pd.read_json(path_aux, lines=True)
  File "/home/gonzalo/anaconda3/lib/python3.7/site-packages/pandas/io/json/json.py", line 421, in read_json
    lines=lines, chunksize=chunksize, compression=compression,
  File "/home/gonzalo/anaconda3/lib/python3.7/site-packages/pandas/io/json/json.py", line 471, in __init__
    self.data = self._preprocess_data(data)
  File "/home/gonzalo/anaconda3/lib/python3.7/site-packages/pandas/io/json/json.py", line 482, in _preprocess_data
    data = data.read()
  File "/home/gonzalo/anaconda3/

KeyboardInterrupt: 

In [ ]:
#Load total bases
df_total_bases = pre.total_bases_dataset(data_path)

In [ ]:
#Calculate the occupation rate
df_occupation_rate = pre.make_df_occupation_rate(df_dock_bikes, df_total_bases)

In [ ]:
#Let's see the statistics
df_total_bases.describe()

In [ ]:
#Let's have a look of the head
df_occupation_rate.head()

In [ ]:
#Let's see the shape
df_occupation_rate.shape

#### Let's divide the dataset into train, validaton an the other one for test.

In [ ]:
train = df_occupation_rate.iloc[:-2]
train.tail(1)

In [ ]:
validation = df_occupation_rate[-2:-1]
validation

In [ ]:
test = df_occupation_rate[-1:]
test

#### Kindly note that the split is just informative, we will split the dataset before train the model. In the following engineering we will use the train with validations because it is needed the day to predict to get some features

## 2. Times feature engineering

The procedure we are going to follow is the following:

- Make the feature day_month, since it may be important to know if we are at the beginning or end of the month.
- Make the feature back_days, which will be the number of days between the day to predict and the sample.
- Maje the feature weekday which will be divided in two, since in order to keep the information that the sunday is next to monday, we will divide by coordinates where the days of the week will be aroun a circle as a clock.

#### Feature day of month

In [ ]:
#Let's keep the test data (last value) for the test
train = df_occupation_rate.iloc[:-1]
train.head()

In [ ]:
#Check how to get the day of the month
print(train.index[2])
train.index[2].day

In [ ]:
#Let's make a new column with the day of the month
train['day_month'] = train.index.day
train.head()

#### Feature backdays

In [ ]:
#Let's see the day to predict
train.index[-1]

In [ ]:
#Let's define the day to predict
date_to_predict =  train.index[-1]

In [ ]:
#Therefore, the backdays will be:
diff_dates = date_to_predict - train.index

In [ ]:
#And now, it is defined the column with such difference
train['back_days'] = diff_dates.days
train.tail(5)

#### Feature weekday

In [ ]:
#Let's check how to get the day of the week
train.index[2].weekday()

In [ ]:
#Now, we define the coordinates of the each day of the week like a clock

days_of_week = 7

train['sin_weekday'] = np.sin(2*np.pi*train.index.weekday/days_of_week)
train['cos_weekday'] = np.cos(2*np.pi*train.index.weekday/days_of_week)

train.head(14)

In [ ]:
#Finally, we define a function for all the times feature engineering
def make_time_features(df):
    date_to_predict =  df.index[-1]
    diff_dates = date_to_predict - df.index
    df['day_month'] = df.index.day
    df['back_days'] = diff_dates.days
    df['sin_weekday'] = np.sin(2*np.pi*df.index.weekday/7)
    df['cos_weekday'] = np.cos(2*np.pi*df.index.weekday/7)
    return df    

## 3. Lag Variables

In this case we will create a variable of how the occuppation rate was yesterday, 7 days before, and 30 days before.

In [ ]:
#Let's define the function to create such columns
def lag_function(df, column):
    df[column + '_' + 'lag' + '_' + '1'] = df[column].shift(1)
    df[column + '_' + 'lag' + '_' + '7'] = df[column].shift(7)
    df[column + '_' + 'lag' + '_' + '30'] = df[column].shift(30)
    return df

In [ ]:
#Filter the columns with the columns we need thed data
columns = [x for x in train if 'OccupationRate' in x]
columns

In [ ]:
#Now we apply the functio for each postal code
for column in columns:
    train = lag_function(train, column)

In [ ]:
#Let's take a look of the train
train.tail()

#### As the first days do not have data for the lag, it is decided to fill such NaN with 0, in that way we do not lie to the model

In [ ]:
train.fillna(0, inplace=True)
train.head(31)

In [ ]:
#Let's define function for create the lag variables
def bucle_lag_function(df):
    columns = [x for x in df if 'OccupationRate' in x]
    for column in columns:
        df = lag_function(df, column)
    df.fillna(0, inplace=True)
    return df

## 4. Weather feature engineering

We are going to use the weather data from AEMET for predicting. In this case, we will use rain, temperature, wind and presure, since they seem the kind of features that can help to predict the usage of the bikes

In [ ]:
#Open the meteo data
df_weather = pd.read_json(os.path.join(data_path, 'METEO.txt'))
df_weather.head()

In [ ]:
#Make the date column to datetime type
df_weather['fecha'] = pd.to_datetime(df_weather['fecha'], format='%Y-%m-%d')
df_weather.info()

In [ ]:
#Choose the columns we want to add to the data set, besides the date
selected_columns = ['fecha', 'prec', 'presMax', 'tmed', 'velmedia']

In [ ]:
#Filter the weather data set with such columns
df_weather_selected = df_weather[selected_columns]
df_weather_selected.info()

#### We have found a problem, as the decimal separetor of the data is coma and not point pandas do not read such data as a number. Therefore, let's define a function to change such type of data

In [ ]:
def convert_to_number(x):
    try:
        x = x.replace(',','.')
    except:
        x = 0
    try:
        x = float(x)
    except:
        x = 0
    return x

In [ ]:
#And let's apply such function to each column but fecha (date)
for column in [x for x in selected_columns if 'fecha' not in x]:
    print(column)
    df_weather_selected[column] = df_weather_selected[column].apply(convert_to_number)

In [ ]:
#Let's see now the types of data
df_weather_selected.info()

In [ ]:
#Now, we can merge the weather data with the dataset
train = train.merge(df_weather_selected[['fecha', 'prec', 'presMax', 'tmed', 'velmedia']], how='left', left_index=True, right_on='fecha')
train.drop('fecha', axis=1, inplace=True)
train.reset_index(drop=True, inplace=True)
train.head()

In [ ]:
#As it is seen, somedays it is not recorded the weather data.
#It is been decided to fill such data with the value of the last day, since it is the most likely to happen.
train[['prec', 'presMax', 'tmed', 'velmedia']].fillna(method='ffill', inplace=True)
train[['prec', 'presMax', 'tmed', 'velmedia']]

In [ ]:
#Let's take a look of the dataset
train

In [ ]:
#Finally, let's define the function for add the weather data
def make_weather_features(df, df_weather):
    df_weather['fecha'] = pd.to_datetime(df_weather['fecha'], format='%Y-%m-%d')
    selected_columns = ['fecha', 'prec', 'presMax', 'tmed', 'velmedia']
    df_weather_selected = df_weather[selected_columns]
    for column in [x for x in selected_columns if 'fecha' not in x]:
        df_weather_selected[column] = df_weather_selected[column].apply(convert_to_number)
    df = df.merge(df_weather_selected[['fecha', 'prec', 'presMax', 'tmed', 'velmedia']], how='left', left_index=True, right_on='fecha')
    df.drop('fecha', axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df[['prec', 'presMax', 'tmed', 'velmedia']].fillna(method='ffill', inplace=True)
    return df

## 5. The function for all the feature engineering

In [ ]:
#Let's define a function to do all the feature engineering
def feature_engineering(df, df_weather):
    df = make_time_features(df)
    df = bucle_lag_function(df)
    df = make_weather_features(df, df_weather)
    return df

In [ ]:
#Load data for check the function
df_dock_bikes = pre.make_all_dataset(data_path, 'dock_bikes',by_postal_code=True, verbose=False )
df_total_bases = pre.total_bases_dataset(data_path)
df_occupation_rate = pre.make_df_occupation_rate(df_dock_bikes, df_total_bases)
df_test_fun_feature_engineering = df_occupation_rate[:-1]
df_test_fun_feature_engineering.head()

In [ ]:
#Load the data to check the function
df_weather_test_fun_feature_engineering = pd.read_json(os.path.join(data_path, 'METEO.txt'))
df_weather_test_fun_feature_engineering.head(1)

In [ ]:
#Check the function
train_test_fun_feature_engineering = feature_engineering(df_test_fun_feature_engineering, df_weather_test_fun_feature_engineering)

In [ ]:
#The data should be as the same as train dataset
np.sum(train_test_fun_feature_engineering.values - train.values)

## 6. Model

### A. First Model

In [ ]:
#Choose the columns that correspond to the features
columns = train.columns
X_columns = [x for x in columns if '28' in x and 'lag' in x]
X_columns = X_columns + ['prec', 'presMax', 'tmed', 'velmedia', 'day_month', 'back_days', 'sin_weekday', 'cos_weekday']
X_columns

In [ ]:
#Choose the columns that correspond to the labels
y_columns = [y for y in columns if '28' in y and 'lag' not in y]
y_columns

In [ ]:
#Get rid of the validation data
X = train.iloc[:-1][X_columns]
y = train.iloc[:-1][y_columns]

In [ ]:
#Check sizes of the project
print(X.shape)
print(y.shape)

In [ ]:
#Let's define a function to separate the labels and featueres
def get_X_y(df):
    columns = df.columns
    X_columns = [x for x in columns if '28' in x and 'lag' in x]
    X_columns = X_columns + ['prec', 'presMax', 'tmed', 'velmedia', 'day_month', 'back_days', 'sin_weekday', 'cos_weekday']
    y_columns = [y for y in columns if '28' in y and 'lag' not in y]
    X = df[X_columns]
    y = df[y_columns]
    return (X, y)

In [ ]:
#Import random forest
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#Let's define the first model
model = RandomForestRegressor(max_depth=4, n_estimators=50, random_state=20)

In [ ]:
#Fit the model
model.fit(X, y)

In [ ]:
#Define the validation data
X_validation = train.iloc[-1:][X_columns]
y_real = train.iloc[-1:][y_columns]

In [ ]:
#Check size of validation data
print(X_validation.shape)
print(y_real.shape)

In [ ]:
#Get the prediction of the validation data
y_predicted_validation = model.predict(X_validation)
y_predicted_validation

In [ ]:
#Import libraries for metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
#Function to get all the metrics together
def metrics(y_real, y_predicted):
    mae = mean_absolute_error(y_real, y_predicted)
    print('MAE: %f' % mae)
    mse = mean_squared_error(y_real, y_predicted)
    print('MSE: %f' % mse)
    rmse = sqrt(mse)
    print('RMSE: %f' % rmse)
    
    return mae, mse, rmse

In [ ]:
#Get metrics
metrics(y_real, y_predicted_validation)

### B. RandomizedSearchCV Model

Now,let's choose the best hyperparametres

In [ ]:
#Library
from sklearn.model_selection import GridSearchCV

In [ ]:
#Define the model
GridedModel = GridSearchCV(RandomForestRegressor(),
                                    param_grid ={'n_estimators' : [10,30,50,80,100,130,150],
                                                        'max_depth' : [5,10,15,20,25,30,35,40]},
                                    scoring='neg_mean_squared_error',
                                    n_jobs=-1, verbose=2)

In [ ]:
#Get the X and y for the grid search
X_GridSearch, y_GridSearch = get_X_y(train)
print(X_GridSearch.shape, y_GridSearch.shape)

In [ ]:
#Fit the model
GridedModel.fit(X_GridSearch, y_GridSearch)

In [ ]:
#Look the best parametrers
GridedModel.best_params_

In [ ]:
#See the best score
GridedModel.best_score_

In [ ]:
#Save the best model
model_hiper = GridedModel.best_estimator_

### C. Test

In [ ]:
#Load the data including the test
df_dock_bikes = pre.make_all_dataset(data_path, 'dock_bikes',by_postal_code=True, verbose=False )
df_total_bases = pre.total_bases_dataset(data_path)
df_occupation_rate = pre.make_df_occupation_rate(df_dock_bikes, df_total_bases)
df_weather = pd.read_json(os.path.join(data_path, 'METEO.txt'))
df_test = feature_engineering(df_occupation_rate, df_weather)
print(df_test.shape)
df_test.tail(3)

In [ ]:
#Save the test data
X_test, y_test_real = get_X_y(df_test.iloc[-1:])

In [ ]:
#Check shapes
print(X_test.shape)
print(y_test_real.shape)

In [ ]:
y_test_predicted = model_hiper.predict(X_test)

In [ ]:
metrics(y_test_real, y_test_predicted)

In [ ]:
#Save model
import pickle
pickle.dump(model_hiper, open('model', 'wb'))

## 7. Visualization

In [10]:
#Libraries
import altair as alt
alt.renderers.enable('notebook')
import json
import geopandas as gpd

In [11]:
#Let's open the data of the districts of Madrid
with open(os.path.join(data_path, 'distritos.txt')) as f:
    data = f.readlines()
    data = data[0]
    data = json.loads(data)
    f.close()

In [13]:
#Define a function where add a column saying if such district has bicimad stations
def check_barrio(barrio):
    barrios_con_estacion = ['MONC','RET','TETU','CENT','CHAMB','CHAMA','SALAM','ARGANZ']
    for patron in barrios_con_estacion:
        if patron in barrio.upper():
            return 'YES'
    return 'NOT'

In [14]:
#Add such column (if the district has bicimad stations)
df_distritos = gpd.GeoDataFrame.from_features((data))
df_distritos['barrios_on'] = df_distritos['nombre'].apply(check_barrio)
df_distritos_on = df_distritos[df_distritos['barrios_on'] == 'YES']
df_distritos_on = df_distritos_on[~df_distritos['nombre'].str.contains("cloa")]
distritos = json.loads(df_distritos.to_json())
distritos_on = json.loads(df_distritos_on.to_json())

/home/gonzalo/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:471: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


In [15]:
df_distritos_on

,cartodb_id,coddistrit,created_at,geometry,nombre,shape_area,shape_len,updated_at,barrios_on
6,3,3,2015-07-06T13:59:20Z,"(POLYGON ((-3.664095 40.408523, -3.665152 40.4...",Retiro,5.465317e+06,9523.926778,2015-07-06T13:59:20Z,YES
8,6,6,2015-07-06T13:59:20Z,"(POLYGON ((-3.697636 40.474539, -3.697495 40.4...",Tetuán,5.388122e+06,9925.073151,2015-07-06T13:59:20Z,YES
9,1,1,2015-07-06T13:59:20Z,"(POLYGON ((-3.693162 40.407345, -3.693202 40.4...",Centro,5.231267e+06,10304.545144,2015-07-06T13:59:20Z,YES
10,7,7,2015-07-06T13:59:20Z,"(POLYGON ((-3.691222 40.44619, -3.691786 40.44...",Chamberí,4.674107e+06,9020.565056,2015-07-06T13:59:20Z,YES
11,5,5,2015-07-06T13:59:20Z,"(POLYGON ((-3.673616 40.482705, -3.673682 40.4...",Chamartín,9.170135e+06,13401.482255,2015-07-06T13:59:20Z,YES
12,4,4,2015-07-06T13:59:20Z,"(POLYGON ((-3.659399 40.438273, -3.659592 40.4...",Salamanca,5.387252e+06,10862.998823,2015-07-06T13:59:20Z,YES
13,2,2,2015-07-06T13:59:20Z,"(POLYGON ((-3.703886 40.405197, -3.702973 40.4...",Arganzuela,6.466406e+06,12834.885747,2015-07-06T13:59:20Z,YES


In [ ]:
#Plot which districts has bicimad stations
source = alt.Data(values=distritos['features'])

Madrid = alt.Chart(source).mark_geoshape(
        stroke='black',
        strokeWidth=1
).encode().properties( title="Madrid City"
    )

barrios_plot = alt.Chart(source).mark_geoshape(
        fill='lightgray',
        stroke='black'
    ).encode(
        alt.Color('properties.barrios_on',
                  type='ordinal',
                  scale=alt.Scale(scheme='tableau10'),
                  title='Neighborhood with bike stations')
    )

Madrid + barrios_plot

In [ ]:
#Let's pivos the data for the plot
def pivot_data_frame(df):
    df_pivot = pd.DataFrame(columns=['Postal Code', 'Date', 'Occupation Rate'])

    columns = df.columns
    index = df.index

    for column in columns:
        for row in index:
            aux_dict = {'Postal Code' : [row],
                        'Date' : [column],
                        'Occupation Rate' : [df.loc[row, column]]
                       }
            df_aux = pd.DataFrame(aux_dict)
            df_pivot = pd.concat([df_pivot, df_aux])
    df_pivot.reset_index(inplace=True, drop=True)
    return df_pivot

In [ ]:
#Load the data for the plot
df_dock_bikes = pre.make_all_dataset(data_path, 'dock_bikes',by_postal_code=True, verbose=False )
df_total_bases = pre.total_bases_dataset(data_path)
df_occupation_rate = pre.make_df_occupation_rate(df_dock_bikes, df_total_bases)
df_weather = pd.read_json(os.path.join(data_path, 'METEO.txt'))
df_plot = feature_engineering(df_occupation_rate, df_weather)
columns = [x for x in df_plot.columns if '28' in x and not 'lag' in x]
df_plot = df_plot[columns]
df_plot.columns = [x.split('_')[0] for x in df_plot.columns]
df_plot.index = df_occupation_rate.index
df_plot = df_plot.T
df_plot = pivot_data_frame(df_plot)
df_coordinates_postal_codes = pd.read_csv(os.path.join(data_path, 'CoordinatesPostalCodes.csv'), sep=';', dtype={'Codigo Postal' : str})
df_coordinates_postal_codes.set_index('Codigo Postal', inplace=True)
df_plot = df_plot.merge(df_coordinates_postal_codes, how='left', left_on='Postal Code', right_index=True)

In [ ]:
#Choose prediction
df_plot_prediction = df_plot[df_plot['Date'] == df_plot['Date'].max()]
df_plot_prediction

In [ ]:
#Add the number of stations by postal code
stations_postal_code = pre.load_dict_from_file(data_path)

df_stations_postal_code = pd.DataFrame()
list_aux = []

for postal_code in stations_postal_code:
    count_stations = len(stations_postal_code[postal_code])
    list_aux.append(count_stations)
    df_stations_postal_code[postal_code] = list_aux
    list_aux = []
    
df_stations_postal_code = df_stations_postal_code.T
df_stations_postal_code.columns = ['n_stations']
df_plot = df_plot.merge(df_stations_postal_code, how='left', left_on='Postal Code', right_index=True)
df_plot_prediction = df_plot_prediction.merge(df_stations_postal_code, how='left', left_on='Postal Code', right_index=True)

In [ ]:
#Plot the dashboard

source = alt.Data(values=distritos['features'])
source_on = alt.Data(values=distritos_on['features'])

width = 750
height = 500

postal_codes = list(set(df_plot['Postal Code']))
postal_codes.sort()
postal_codes_selection = alt.binding_select(options=postal_codes)
slider_selection = alt.selection_single(bind=postal_codes_selection,
                                        fields=['Postal Code'],
                                        name="Postal Codes")

Madrid = alt.Chart(source).mark_geoshape(
        stroke='black',
        strokeWidth=1
).encode().properties(title="Madrid City",
                      width=width/2,
                      height=height/2,
    )

barrios_plot = alt.Chart(source).mark_geoshape(
        fill='lightgray',
        stroke='black'
    ).encode(
        alt.Color('properties.barrios_on',
                  type='ordinal',
                  scale=alt.Scale(scheme='tableau10'),
                  title='Neighborhood with bike stations',
                 legend=alt.Legend(orient="left"))
    )

barrios_on = alt.Chart(source_on).mark_geoshape(
        stroke='black'
    ).encode().properties(title="Prediction",
                          width=width/2,
                          height=height/2)

points = alt.Chart(df_plot).mark_circle().encode(
    longitude='Longitud:Q',
    latitude='Latitud:Q',
    size=alt.Size('n_stations:Q', title='Number of Stations'),
    color=alt.Color('Occupation Rate:Q', scale=alt.Scale(scheme="spectral"), title="Occupation Rate"),
    tooltip='Occupation Rate:Q'
).add_selection(slider_selection).transform_filter(slider_selection)

labels = alt.Chart(df_plot_prediction).mark_text(baseline='top'
     ).encode(
         longitude='Longitud:Q',
         latitude='Latitud:Q',
         text='Postal Code:O',
         size=alt.value(10),
         opacity=alt.value(1)
     ).transform_filter(slider_selection)

time_serie = alt.Chart(df_plot).mark_line().encode(
    x='Date:T',
    y='Occupation Rate:Q',
    tooltip='Occupation Rate:Q'
).transform_filter(slider_selection
).properties(title='Evolution', width=width, height=height/2
)

(Madrid + barrios_plot | barrios_on + points + labels) & time_serie

In [ ]:
def plot(df_plot, df_plot_prediction, data_path):
    
    with open(os.path.join(data_path, 'distritos.txt')) as f:
        data = f.readlines()
        data = data[0]
        data = json.loads(data)
        f.close()
    
    df_distritos = gpd.GeoDataFrame.from_features((data))
    df_distritos['barrios_on'] = df_distritos['nombre'].apply(check_barrio)
    df_distritos_on = df_distritos[df_distritos['barrios_on'] == 'YES']
    df_distritos_on = df_distritos_on[~df_distritos['nombre'].str.contains("cloa")]
    distritos = json.loads(df_distritos.to_json())
    distritos_on = json.loads(df_distritos_on.to_json())
    
    source = alt.Data(values=distritos['features'])
    source_on = alt.Data(values=distritos_on['features'])

    width = 750
    height = 500

    postal_codes = list(set(df_plot['Postal Code']))
    postal_codes.sort()
    postal_codes_selection = alt.binding_select(options=postal_codes)
    slider_selection = alt.selection_single(bind=postal_codes_selection,
                                            fields=['Postal Code'],
                                            name="Postal Codes")

    Madrid = alt.Chart(source).mark_geoshape(
            stroke='black',
            strokeWidth=1
    ).encode().properties(title="Madrid City",
                          width=width/2,
                          height=height/2,
        )

    barrios_plot = alt.Chart(source).mark_geoshape(
            fill='lightgray',
            stroke='black'
        ).encode(
            alt.Color('properties.barrios_on',
                      type='ordinal',
                      scale=alt.Scale(scheme='tableau10'),
                      title='Neighborhood with bike stations',
                     legend=alt.Legend(orient="left"))
        )

    barrios_on = alt.Chart(source_on).mark_geoshape(
            stroke='black'
        ).encode().properties(title="Prediction",
                              width=width/2,
                              height=height/2)

    points = alt.Chart(df_plot).mark_circle().encode(
        longitude='Longitud:Q',
        latitude='Latitud:Q',
        size=alt.Size('n_stations:Q', title='Number of Stations'),
        color=alt.Color('Occupation Rate:Q', scale=alt.Scale(scheme="spectral"), title="Occupation Rate"),
        tooltip='Occupation Rate:Q'
    ).add_selection(slider_selection).transform_filter(slider_selection)

    labels = alt.Chart(df_plot_prediction).mark_text(baseline='top'
         ).encode(
             longitude='Longitud:Q',
             latitude='Latitud:Q',
             text='Postal Code:O',
             size=alt.value(10),
             opacity=alt.value(1)
         ).transform_filter(slider_selection)

    time_serie = alt.Chart(df_plot).mark_line().encode(
        x='Date:T',
        y='Occupation Rate:Q',
        tooltip='Occupation Rate:Q'
    ).transform_filter(slider_selection
    ).properties(title='Evolution', width=width, height=height/2
    )

    return (Madrid + barrios_plot | barrios_on + points + labels) & time_serie

### New Plot with postal code

In [16]:
#Let's open the data of the districts of Madrid
with open(os.path.join(data_path, 'MADRID.geojson')) as f:
    postal_codes_geo = f.read().splitlines()
    postal_codes_geo = "".join(postal_codes_geo)
    f.close()

In [17]:
#Convert string to dict
postal_codes_geo_dict = json.loads(postal_codes_geo)

#Convert dict into dataframe
df_postal_codes = gpd.GeoDataFrame.from_features((postal_codes_geo_dict))
df_postal_codes.head()

,ALTA_DB,CODIGO_INE,COD_POSTAL,ID_CP,geometry
0,2015/07/09,28079,28008,2.807900e+11,POLYGON ((-3.718763372131946 40.43541336533451...
1,2015/07/09,28082,28410,2.808200e+11,POLYGON ((-3.800949402992558 40.71124645625895...
2,2015/07/09,28003,28749,2.800300e+11,POLYGON ((-3.808660970827134 40.88823418785745...
3,2015/07/09,28173,28598,2.817300e+11,POLYGON ((-3.257728660948942 40.05274889304695...
4,2015/07/09,28179,28512,2.817900e+11,POLYGON ((-3.288194205198408 40.33271289928086...


In [ ]:
#Function for filtering which postal code has stations

def check_postal_code(cod_postal):
    stations_postal_code_list = ['28013','28004','28015','28005','28008','28014','28012','28001','28009','28007','28006','28046','28045','28010','28003','28020','28002','28036','28016']
    for postal_code in stations_postal_code_list:
        if postal_code == cod_postal:
            return 'YES'
    return 'NOT'

In [ ]:
#Add column if such postal code has bike stations
df_postal_codes['pc_on'] = df_postal_codes['COD_POSTAL'].apply(check_postal_code)
#Create a dataframe just with the postal codes that have bike stations
df_postal_codes_on = df_postal_codes[df_postal_codes['pc_on'] == 'YES']

In [ ]:
postal_codes_dict = json.loads(df_postal_codes.to_json())
postal_codes_dict_on = json.loads(df_postal_codes_on.to_json())

In [ ]:
#Plot which districts has bicimad stations
source = alt.Data(values=postal_codes_dict['features'])

Madrid = alt.Chart(source).mark_geoshape(
        stroke='black',
        strokeWidth=0.5
).encode(
        alt.Color('properties.pc_on',
                  type='ordinal',
                  scale=alt.Scale(scheme='tableau10'),
                  title='Postal Codes with bike stations')
).properties( title="Madrid Postal Codes"
    )

Madrid

In [ ]:
#Load the data for the new plot
df_dock_bikes = pre.make_all_dataset(data_path, 'dock_bikes',by_postal_code=True, verbose=False )
df_total_bases = pre.total_bases_dataset(data_path)
df_occupation_rate = pre.make_df_occupation_rate(df_dock_bikes, df_total_bases)
df_weather = pd.read_json(os.path.join(data_path, 'METEO.txt'))
df_plot = feature_engineering(df_occupation_rate, df_weather)
columns = [x for x in df_plot.columns if '28' in x and not 'lag' in x]
df_plot = df_plot[columns]
df_plot.columns = [x.split('_')[0] for x in df_plot.columns]
df_plot.index = df_occupation_rate.index
df_plot = df_plot.T
df_plot_predic = pd.DataFrame(df_plot[df_plot.columns.max()])
df_plot_predic.columns = ['prediction']
df_postal_codes_on = df_postal_codes_on.merge(df_plot_predic, left_on='COD_POSTAL', right_index=True)
postal_codes_dict_on = json.loads(df_postal_codes_on.to_json())

In [ ]:
source = alt.Data(values=postal_codes_dict['features'])
source2 = alt.Data(values=postal_codes_dict_on['features'])

pcode = alt.selection(type="single", fields=['Postal Code'])

Madrid = alt.Chart(source).mark_geoshape(
        stroke='black',
        strokeWidth=0.5
).encode(
    alt.Color('properties.pc_on',
              type='ordinal',
              scale=alt.Scale(scheme='tableau10'),
              title='Postal Codes with bike stations',
              legend=alt.Legend(orient="left"))
).properties(title="Madrid Postal Codes"
)

Areas = alt.Chart(source2).mark_geoshape(
    stroke='black',
    strokeWidth=0.5
).encode(
    alt.Color('properties.prediction:Q',
              scale=alt.Scale(scheme='redyellowgreen'))
).properties(title="Prediction"
).add_selection(pcode)

time_serie = alt.Chart(df_plot).mark_line().encode(
    x='Date:T',
    y='Occupation Rate:Q',
    color = alt.Color('Postal Code:N',
              scale=alt.Scale(scheme='tableau10')),
    tooltip='Occupation Rate:Q'
).properties(title='Evolution', width=width, height=height/2
).transform_filter(pcode)

(Madrid | Areas) & time_serie